# This notebook ...

In [1]:
import pathlib
import sys
import yaml

import pandas as pd
import torch
import torch.optim as optim
import mlflow
import optuna
import joblib

## Read config

In [2]:
with open(pathlib.Path('.').absolute().parent.parent / "config.yml", "r") as file:
    config = yaml.safe_load(file)

## Import virtual_stain_flow software 

In [3]:
sys.path.append(config['paths']['software_path'])
print(str(pathlib.Path('.').absolute().parent.parent))

## Dataset
from virtual_stain_flow.datasets.PatchDataset import PatchDataset
from virtual_stain_flow.datasets.CachedDataset import CachedDataset

## FNet training
from virtual_stain_flow.models.fnet import FNet
from virtual_stain_flow.trainers.Trainer import Trainer

## wGaN training
from virtual_stain_flow.models.unet import UNet
from virtual_stain_flow.models.discriminator import GlobalDiscriminator
from virtual_stain_flow.trainers.WGaNTrainer import WGaNTrainer

## wGaN losses
from virtual_stain_flow.losses.GradientPenaltyLoss import GradientPenaltyLoss
from virtual_stain_flow.losses.DiscriminatorLoss import DiscriminatorLoss
from virtual_stain_flow.losses.GeneratorLoss import GeneratorLoss

from virtual_stain_flow.transforms.MinMaxNormalize import MinMaxNormalize
from virtual_stain_flow.transforms.PixelDepthTransform import PixelDepthTransform

## Metrics
from virtual_stain_flow.metrics.MetricsWrapper import MetricsWrapper
from virtual_stain_flow.metrics.PSNR import PSNR
from virtual_stain_flow.metrics.SSIM import SSIM

## callback
from virtual_stain_flow.callbacks.MlflowLogger import MlflowLogger
from virtual_stain_flow.callbacks.IntermediatePlot import IntermediatePatchPlot

/home/weishanli/Waylab/pediatric_cancer_atlas_analysis


## Define paths and other train parameters

In [4]:
## Loaddata for train
LOADDATA_FILE_PATH = pathlib.Path('.').absolute().parent.parent \
    / '0.data_preprocessing' / 'data_split_loaddata' / 'loaddata_train.csv'
assert LOADDATA_FILE_PATH.exists()
SC_FEATURES_DIR = pathlib.Path(config['paths']['sc_features_path'])

## Output directories
MLFLOW_DIR = pathlib.Path('.').absolute() / 'optuna_mlflow'
MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

OPTUNA_JOBLIB_DIR = pathlib.Path('.').absolute() / 'optuna_joblib'
OPTUNA_JOBLIB_DIR.mkdir(parents=True, exist_ok=True)

## Basic data generation, model convolutional depth and max epoch definition
PATCH_SIZE = 256
DEPTH = 5
EPOCHS = 1_000

## Channels for input and target are read from config
INPUT_CHANNEL_NAMES = config['data']['input_channel_keys']
TARGET_CHANNEL_NAMES = config['data']['target_channel_keys']

## Defines how the train data will be divided to train models on two levels of confluence

In [ ]:
DATA_GROUPING = {
    'high_confluence': {
        'seeding_density': [12_000, 8_000]
    },
    'low_confluence': {
        'seeding_density': [4_000, 2_000, 1_000]
    }
}

## Define optimization objectives

In [5]:
import gc
def free_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()

def objective(trial, dataset, channel_name, confluence_group_name):
    
    ## Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # adam optimizer learning rate
    beta0 = trial.suggest_float('beta0', 0.5, 0.9) # adam optimizer beta 0
    beta1 = trial.suggest_float('beta1', 0.9, 0.999) # adam optimizer beta 1
    betas = (beta0, beta1)

    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    patience = trial.suggest_int('patience', 5, 20) # early stop patience

    conv_depth = trial.suggest_int('conv_depth', 3, 5) # convolutional depth for fnet model

    ## Setup model and optimizer
    model = FNet(depth=conv_depth)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas)
    
    ## Metrics to be computed (and logged)
    metric_fns = {
        "mse_loss": MetricsWrapper(_metric_name='mse', module=torch.nn.MSELoss()),
        "ssim_loss": SSIM(_metric_name="ssim"),
        "psnr_loss": PSNR(_metric_name="psnr"),
    }

    ## Params to log with mlflow
    params = {
            "lr": lr,
            "beta0": beta0,
            "beta1": beta1,
            "depth": conv_depth,
            "patch_size": PATCH_SIZE,
            "batch_size": batch_size,
            "epochs": EPOCHS,
            "patience": patience,
            "channel_name": channel_name,
        }

    ## mlflow logger callback
    mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name=f'FNet_optimize_{confluence_group_name}',
        mlflow_start_run_args={'run_name': f'FNet_optimize_{confluence_group_name}_{channel_name}', 'nested': True},
        mlflow_log_params_args=params
    )
    
    ## Trainer
    trainer = Trainer(
        model = model,
        optimizer = optimizer,
        backprop_loss = torch.nn.L1Loss(), # MAE loss for backpropagation
        dataset = dataset,
        batch_size = batch_size,
        epochs = EPOCHS,
        patience = patience,
        callbacks=[mlflow_logger_callback],
        metrics=metric_fns,
        device = 'cuda',
        early_termination_metric='L1Loss'
    )

    # Train the model and log validation loss
    trainer.train()
    val_loss = trainer.best_loss

    del model
    del optimizer
    del metric_fns
    del mlflow_logger_callback
    del trainer
    
    free_gpu_memory()

    return val_loss

In [7]:
RUN_EXTRA = False
loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)

for confluence_group_name, conditions in DATA_GROUPING.items():

    ## Load dataset
    loaddata_condition_df = loaddata_df.copy()
    for condition, values in conditions.items():
        loaddata_condition_df = loaddata_condition_df[
            loaddata_condition_df[condition].isin(values)
        ]

    sc_features = pd.DataFrame()
    for plate in loaddata_condition_df['Metadata_Plate'].unique():
        sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
        if not sc_features_parquet.exists():
            print(f'{sc_features_parquet} does not exist, skipping...')
            continue 
        else:
            sc_features = pd.concat([
                sc_features, 
                pd.read_parquet(
                    sc_features_parquet,
                    columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
                )
            ])

    ## Create patch dataset
    pds = PatchDataset(
        _loaddata_csv=loaddata_condition_df,
        _sc_feature=sc_features,
        _input_channel_keys=INPUT_CHANNEL_NAMES,
        _target_channel_keys=TARGET_CHANNEL_NAMES,
        _input_transform=PixelDepthTransform(src_bit_depth=16, target_bit_depth=8, _always_apply=True),
        _target_transform=MinMaxNormalize(_normalization_factor=(2 ** 16) - 1, _always_apply=True),
        patch_size=PATCH_SIZE,
        verbose=False,
        patch_generation_method="random_cell",
        patch_generation_random_seed=42
    )

    for channel_name in TARGET_CHANNEL_NAMES:

        ## Cache dataset of channel
        pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
        pds.set_target_channel_keys(channel_name)
        cds = CachedDataset(
            dataset=pds,
            prefill_cache=True
        )

        print(f"Beginning optimization for channel: {channel_name} for {confluence_group_name}")

        # Load the existing study for the current channel
        study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_{confluence_group_name}.joblib"
        if study_path.exists():
            if RUN_EXTRA:
                study = joblib.load(study_path)
            else:
                print("Skipping optimization due to existing joblib...")
                continue
        else:
            study = optuna.create_study(
                direction="minimize",
                study_name=f"FNet_optimize_{channel_name}_{confluence_group_name}",
                sampler=optuna.samplers.TPESampler(seed=42)
            )

        # Optimize the objective function for the current channel
        study.optimize(lambda trial: objective(trial, cds, channel_name, confluence_group_name), n_trials=50)

        # Save the updated study for the current channel
        joblib.dump(study, study_path)

        # Print best trial results
        print(f"Best trial for channel {channel_name}:")
        print(f"  Validation Loss: {study.best_trial.value}")
        print(f"  Hyperparameters: {study.best_trial.params}")

[I 2025-02-20 12:46:38,697] A new study created in memory with name: FNet_optimize_OrigDNA_high_confluence


Beginning optimization for channel: OrigDNA for high_confluence


2025/02/20 12:46:39 INFO mlflow.tracking.fluent: Experiment with name 'FNet_optimize_high_confluence' does not exist. Creating a new experiment.


Early termination at epoch 99 with best validation metric 0.025294281542301178


[I 2025-02-20 12:47:49,143] Trial 0 finished with value: 0.025294281542301178 and parameters: {'lr': 0.0001329291894316216, 'beta0': 0.8802857225639664, 'beta1': 0.9724674002393291, 'batch_size': 48, 'patience': 7, 'conv_depth': 3}. Best is trial 0 with value: 0.025294281542301178.


...

Early termination at epoch 85 with best validation metric 0.008079479836548368


[I 2025-02-21 02:18:57,053] Trial 48 finished with value: 0.008079479836548368 and parameters: {'lr': 0.00033198967374893103, 'beta0': 0.7783547221751052, 'beta1': 0.9332292552852137, 'batch_size': 16, 'patience': 16, 'conv_depth': 4}. Best is trial 46 with value: 0.007608745014294982.


Early termination at epoch 91 with best validation metric 0.01106931579609712


[I 2025-02-21 02:20:35,424] Trial 49 finished with value: 0.01106931579609712 and parameters: {'lr': 0.00030384226118136134, 'beta0': 0.7412980394447174, 'beta1': 0.9349063637927016, 'batch_size': 16, 'patience': 16, 'conv_depth': 4}. Best is trial 46 with value: 0.007608745014294982.


Best trial for channel OrigRNA:
  Validation Loss: 0.007608745014294982
  Hyperparameters: {'lr': 8.683717750721345e-05, 'beta0': 0.6513543410289212, 'beta1': 0.9353240725137701, 'batch_size': 16, 'patience': 14, 'conv_depth': 4}
